## Multivariate colocalization

### Aim:

This notebook aims to demonstrate a workflow of generating posterior inclusion probabilities (PIPs) from GWAS summary statistics using SuSiE linear regression and construsting SNP signal clusters from global eQTL analysis data obtained from multivariate SuSiE models. 

### Input:

1) GWAS Summary Statistics with the following columns:
- chr: chromosome number
- bp: base pair position
- a1: effect allele
- a2: other allele
- beta: effect size
- se: standard error of beta
- z: z score

2) eQTL data from multivariate SuSiE model with the following columns:
- chr: chromosome number
- bp: base pair position
- a1: effect allele
- a2: other allele
- pip: posterior inclusion probability

3) LD correlation matrix 

### Output:

Intermediate files:

1) GWAS PIP file 

2) eQTL annotation file

Final Outputs:

1) Enrichment analysis result prefix.enloc.enrich.rst: estimated enrichment parameters and standard errors.

2) Signal-level colocalization result prefix.enloc.sig.out: the main output from the colocalization analysis with the following format
- column 1: signal cluster name (from eQTL analysis)
- column 2: number of member SNPs
- column 3: cluster PIP of eQTLs
- column 4: cluster PIP of GWAS hits (without eQTL prior)
- column 5: cluster PIP of GWAS hits (with eQTL prior)
- column 6: regional colocalization probability (RCP)

3) SNP-level colocalization result prefix.enloc.snp.out: SNP-level colocalization output with the following form at
- column 1: signal cluster name
- column 2: SNP name
- column 3: SNP-level PIP of eQTLs
- column 4: SNP-level PIP of GWAS (without eQTL prior)
- column 5: SNP-level PIP of GWAS (with eQTL prior)
- column 6: SNP-level colocalization probability

4) Sorted list of colocalization signals with
`sort -grk6 prefix.enloc.sig.out`

### Method:

### Functions:

#### 1) Allele Flip 

Takes into consideration 3 situations: 

1) "Major" and "minor" alleles flipped

2) Different strand but same variant

3) Remove variants with A/T and C/G alleles due to ambiguity

In [ ]:
allele.qc = function(a1,a2,ref1,ref2) {
	# a1 and a2 are the first data-set
	# ref1 and ref2 are the 2nd data-set
	# Make all the alleles into upper-case, as A,T,C,G:
        a1 = toupper(a1)
        a2 = toupper(a2)
        ref1 = toupper(ref1)
        ref2 = toupper(ref2)
	# Strand flip, to change the allele representation in the 2nd data-set
	strand_flip = function(ref) {
		flip = ref
		flip[ref == "A"] = "T"
		flip[ref == "T"] = "A"
		flip[ref == "G"] = "C"
		flip[ref == "C"] = "G"
		flip
	}
	flip1 = strand_flip(ref1)
	flip2 = strand_flip(ref2)
	snp = list()
	# Remove strand ambiguous SNPs (scenario 3)
	snp[["keep"]] = !((a1=="A" & a2=="T") | (a1=="T" & a2=="A") | (a1=="C" & a2=="G") | (a1=="G" & a2=="C"))
	# Remove non-ATCG coding
	snp[["keep"]][ a1 != "A" & a1 != "T" & a1 != "G" & a1 != "C" ] = F
	snp[["keep"]][ a2 != "A" & a2 != "T" & a2 != "G" & a2 != "C" ] = F
	# as long as scenario 1 is involved, sign_flip will return TRUE
	snp[["sign_flip"]] = (a1 == ref2 & a2 == ref1) | (a1 == flip2 & a2 == flip1)
	# as long as scenario 2 is involved, strand_flip will return TRUE
	snp[["strand_flip"]] = (a1 == flip1 & a2 == flip2) | (a1 == flip2 & a2 == flip1)
	# remove other cases, eg, tri-allelic, one dataset is A C, the other is A G, for example.
	exact_match = (a1 == ref1 & a2 == ref2) 
	snp[["keep"]][!(exact_match | snp[["sign_flip"]] | snp[["strand_flip"]])] = F
	return(snp)
}

#### 2) Construct eQTL data frame

In [ ]:
eqtl.split = function(eqtl){
  rows = length(eqtl)
  chr = vector(length = rows)
  pos = vector(length = rows)
  a1 = vector(length = rows)
  a2 = vector(length = rows)
  for (i in 1:rows){
    split1 = str_split(eqtl[i], ":")
    split2 = str_split(split1[[1]][2], "_")
    chr[i]= split1[[1]][1]
    pos[i] = split2[[1]][1]
    a1[i] = split2[[1]][2]
    a2[i] = split2[[1]][3]

  }
  eqtl.df = data.frame(eqtl,chr,pos,a1,a2)
}

#### 3) Remove Duplicate variants 

In [ ]:
remove.dup = function(df){
  df = df %>% arrange(PosGRCh37, -N)
  df = df[!duplicated(df$PosGRCh37),]
  return(df)
}

In [ ]:
greedy = function(alpha){
  len = length(alpha)
  r2 = 0.5
  c_r2 = 1
  ids = vector()
  first = 1
  less = 0
  total = 0
  for(i in 1:len){
    max = max(alpha)
    total = total + max
    idx = which.max(alpha)
    if(length(ids) == 0){
      ids = append(ids,idx)
    }
    
    
    for(j in 1:length(ids)){
      new_r2 = (ld_f2[ids[j],idx])^2
     # print(new_r2)
      if(new_r2 < r2){
        less = 1
        break
      }
    }
    
    if(less == 1){
      break
      }
    
    if(first == 0){
    ids = append(ids,idx)
    }

    alpha = alpha[-idx]
    first = 0
    
  }
  return(list(ids, total))
}

### Workflow:

In [ ]:
library(tidyverse)
library(devtools)
library(susieR)

#### Step 1: Extract common SNPS between the GWAS summary statistics and eQTL data 

In [ ]:
# eqtl
load(file.choose())
eqtl = m_ENSG00000159640

# colnames
load("geneTpmResidualsAgeGenderAdj_rename.ENSG00000159640.transformed_XY.RData")
var = colnames(scaled_ENSG00000159640[[1]])

#ld
ld = readRDS(file.choose())

In [ ]:
gwas_filter = gwas[which(gwas$id %in% var),]

In [ ]:
eqtl.df = eqtl.split(var)

In [ ]:
# allele flip
f_gwas = gwas %>% filter(chr %in% eqtl.df$chr & PosGRCh37 %in% eqtl.df$pos)
eqtl.df.f = eqtl.df %>% filter(pos %in% f_gwas$PosGRCh37)

# check if there are duplicate pos
length(unique(f_gwas$PosGRCh37))

# multiple snps with same pos
dup.pos = f_gwas %>% group_by(PosGRCh37) %>% filter(n() > 1) 

In [ ]:
f_gwas = remove.dup(f_gwas)

qc = allele.qc(f_gwas$testedAllele, f_gwas$otherAllele, eqtl.df.f$a1, eqtl.df.f$a2)
keep = as.data.frame(qc$keep)
sign = as.data.frame(qc$sign_flip)
strand = as.data.frame(qc$strand_flip)

In [ ]:
# sign flip
f_gwas$z[qc$sign_flip] = -1 * f_gwas$z[qc$sign_flip]
f_gwas$testedAllele[qc$sign_flip] = eqtl.df.f$a1[qc$sign_flip]
f_gwas$otherAllele[qc$sign_flip] = eqtl.df.f$a2[qc$sign_flip]

In [ ]:
f_gwas$testedAllele[qc$strand_flip] = eqtl.df.f$a1[qc$strand_flip]
f_gwas$otherAllele[qc$strand_flip] = eqtl.df.f$a2[qc$strand_flip]

In [ ]:
# remove ambigiuous 
if ( sum(!qc$keep) > 0 ) {
  eqtl.df.f = eqtl.df.f[qc$keep,]
  f_gwas = f_gwas[qc$keep,]
}

In [ ]:
# lds filtered
eqtl_id = which(var %in% eqtl.df.f$eqtl)
ld_f = ld[eqtl_id, eqtl_id]
dim(ld_f)

In [ ]:
# ld missing

miss = which(is.na(ld_f), arr.ind=TRUE)


miss_r = unique(as.data.frame(miss)$row)
miss_c = unique(as.data.frame(miss)$col)

total_miss = unique(union(miss_r,miss_c))

if(length(total_miss) != 0){ 
eqtl_id.f = eqtl.df.f[-total_miss,]
}else{eqtl_id.f = eqtl.df.f}

# rename ids

f_gwas = f_gwas %>% mutate(id = paste(f_gwas$chr, paste(f_gwas$PosGRCh37, f_gwas$testedAllele, f_gwas$otherAllele, sep = "_"), sep = ":"))

f_gwas.f = f_gwas %>% filter(id %in% eqtl_id.f$eqtl)


if (length(total_miss)!=0){
ld_f2 = ld_f[-total_miss,]
ld_f2 = ld_f2[,-total_miss]
dim(ld_f2)
}else{ld_f2 = ld_f}

miss = which(is.na(ld_f2), arr.ind=TRUE)

# remove multi-allelic snps that aren't present in both eqtl and gwas 


idx = which(eqtl_id.f$eqtl %notin% f_gwas$id)
if(length(idx) != 0){
eqtl_id.f = eqtl_id.f[-idx,]
ld_f2 = ld_f2[-idx,]
ld_f2 = ld_f2[,-idx]
}
dim(ld_f2)
dim(eqtl_id.f)
dim(f_gwas.f)



#### Step 2: Obtain SNP PIPs from GWAS summary statistics using SuSiE linear regression

In [ ]:
susie_results = susieR::susie_rss(z = f_gwas.f$z,R = ld_f2, check_prior = F)

In [ ]:
susieR::susie_plot(susie_results,"PIP")

In [ ]:
susie_results$z = f_gwas.f$z
susieR::susie_plot(susie_results,"z_original")

#### Step 3: Construct signal clusters using eQTL SNP PIPs

In [ ]:
for( al in 1:10){
  a = greedy(alpha[al,])
  print("cluster")
  print(a)
}

### Minimal Working Example: 